In [3]:
import configparser, os

def qsubSystem(name = "tmpsh", cmd = "echo \'Hello world\'", account = "patil",
               N_nodes = 1, n_cores = 1, mem = "16000M", qname = "1day", 
               filename=None):
    """
    Creates creates SLURM script
    :param name: name of the program, to save bash & error & output
    :param cmd: actual bash cmd
    :param n_cores: cores reserved for job
    :param mem: Ram usage per core in MB
    :param qname: queue name
    :return:
    :rtype: None
    """
  
    #SBATCH -A groupname                # group to which you belong
    #SBATCH -p general                  # partition (queue)
    #SBATCH -N 1                        # number of nodes
    #SBATCH -n 1                        # number of cores
    #SBATCH --mem 100M                  # memory pool for all cores
    #SBATCH -t 0-2:00                   # runtime limit (D-HH:MM)
    #SBATCH -o slurm.%N.%j.out          # STDOUT
    #SBATCH -e slurm.%N.%j.err          # STDERR
    #SBATCH --mail-type=END,FAIL        # notifications for job done & fail
    #SBATCH --mail-user=myemail@embl.de # send-to address

    s = """#!/bin/bash
#SBATCH -A {0}  # group to which you belong
#SBATCH -p {1}  # partition (queue)
#SBATCH -N {2}  # number of nodes
#SBATCH -n {3}  # number of cores
#SBATCH --mem {4}  # memory pool for all cores
#SBATCH -t 1-00:00 # runtime limit (D-HH:MM)
#SBATCH -o $SLURM_SUBMIT_DIR/logs/{5}.%N.%j.out  # STDOUT
#SBATCH -e $SLURM_SUBMIT_DIR/logs/{5}.%N.%j.err  # STDERR
#SBATCH --mail-type=END,FAIL    # notifications for job done & fail    
#SBATCH --mail-user=zeleznia@embl.de # send-to address
mkdir -p ./logs
""".format(account, qname, N_nodes, n_cores, mem, name)

    if filename:
        f = open(filename,'w')
        f.write(s)
        f.write(cmd)
        f.close
    else:
        print(s)
        print(cmd)


config = configparser.ConfigParser()
config.read('config.cfg')


#generating metaphlan
my_cmd="""module load Anaconda3
echo \"Hello world\""""


```{r, engine='bash', count_lines}
```

bin="../../bin/metaphlan2"
data_dir="output_test.tsv"
output_dir="./r_metaphlan"

mkdir -p $output_dir

pattern="fastq"
files=(`cut -f2 $data_dir`)
fLen=${#files[*]}
echo "total files to process with pattern $pattern from $data_dir: $fLen"


if [ -z ${SLURM_ARRAY_TASK_ID+x} ]; then
echo "Setting SLURM_ARRAY_TASK_ID = 1"
SLURM_ARRAY_TASK_ID=1;
else echo "SLURM_ARRAY_TASK_ID is set to '$SLURM_ARRAY_TASK_ID'"; fi

mem=$SLURM_ARRAY_TASK_ID-1
dataset=${files[$mem]}

names=(`cut -f1 $data_dir`)
sample_name=${names[$mem]}
$bin/metaphlan2.py $dataset --bowtie2out $output_dir/$sample_name.bowtie2.bz2 --nproc 4 --input_type fastq > $output_dir/$sample_name"_profile
```



In [8]:
def generateMetaphlan(filename="metaphlan_test.sh"):
    
    #input strings 
    s = "bin=\"{0}\"\n".format(config['metaphlan']['bin'])
    s = s + "data_dir=\"{0}\"\n".format(config['metaphlan']['file_map'])
    s = s + "output_dir=\"{0}\"\n".format(config['metaphlan']['output_dir'])
    s = s + "pattern=\"{0}\"\n".format(config['metaphlan']['pattern'])
    s = s + "files={0}\n".format(config['metaphlan']['files_cmd'])

    cmd_modules="""
module load Anaconda3
"""
    
#defaults
    cmd_array="""
mkdir -p $output_dir

fLen=${#files[*]}
echo "total files to process with pattern $pattern from $data_dir: $fLen"

if [ -z ${SLURM_ARRAY_TASK_ID+x} ]; then
    SLURM_ARRAY_TASK_ID=1;
fi 

if [ -z ${SLURM_NPROCS+x} ]; then
    SLURM_NPROCS=1;
fi

mem=$SLURM_ARRAY_TASK_ID-1
dataset=${files[$mem]}
"""

#metaphlan specific
    my_cmd="""
names=(`cut -f1 $data_dir`)
sample_name=${names[$mem]}
$bin/metaphlan2.py $dataset --bowtie2out $output_dir/$sample_name.bowtie2.bz2 --nproc $SLURM_NPROCS --input_type fastq > $output_dir/$sample_name"_profile"
"""

    my_cmd = s + cmd_array + cmd_modules + my_cmd
    if filename:
        qsubSystem(name=filename,account = "SNIC2017-1-344", n_cores = 4, cmd=my_cmd, filename=filename)
        print ("{0} is done".format(filename))
    else:
        print(my_cmd)
        

    


    
    
    
    
    

In [7]:
generateMetaphlan()

.metaphlan_test is done
